In [45]:
import pandas as pd
import numpy as np
import os
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [46]:
path = r"C:\Users\Alok\Downloads\dataset\espn_data"

In [47]:
def derive_batting_features(batdf, _id):
    col_list=[]
    val = []
    for _team in batdf.team.unique():
        tempdf = batdf[batdf.team == _team]
        tempdf = tempdf.apply(pd.to_numeric, errors='coerce')
        col_list.extend([item%_team for item in ['%s_30+','%s_balls','%s_4s','%s_6s','%s_sr','%s_run_rate']])
        val.append(tempdf[tempdf['r']>=30]['r'].count())
        val.append(tempdf.b.sum())
        val.append(tempdf['4s'].sum())
        val.append(tempdf['6s'].sum())
        val.append(tempdf[tempdf['sr']>100]['sr'].count())
        val.append(tempdf.iloc[0]['run_rate'])
    return dict(zip(["id"]+col_list,[_id]+val))

def derive_bowling_features(bowldf, _id):
    col_list = []
    val = []
    for _team in bowldf.team.unique():
        tempdf = bowldf[bowldf.team == _team]
        tempdf = tempdf.apply(pd.to_numeric, errors='coerce')
        col_list.extend([item%_team for item in ['%s_econ','%s_dots','%s_wd','%s_nb']])
        val.append(tempdf['econ'].mean())
        val.append(tempdf['0s'].sum())
        val.append(tempdf['wd'].sum())
        val.append(tempdf['nb'].sum())
    return dict(zip(["id"]+col_list,[_id]+val))

def check_team1(row):
    if row.result == 'won':
        _list = row.match.split(' v ')
        _list.remove(row.winner)
        if "run" in row.margin:
            return row.winner
        else:
            return _list[0]
    return None

def check_team2(row):
    if row.result == 'won':
        _list = row.match.split(' v ')
        _list.remove(row.winner)
        if "run" in row.margin:
            return _list[0]
        else:
            return row.winner
    return None

In [48]:
finaldf = pd.DataFrame()
for file in os.listdir(path):
    if os.path.isfile(os.path.join(path,file)):
        tempdf = pd.read_csv(os.path.join(path,file))
        list_of_values = []
        tempdf['team1'] = tempdf.apply(lambda row: check_team1(row), axis=1)
        tempdf['team2'] = tempdf.apply(lambda row: check_team2(row), axis=1)
        for index, row in tempdf.iterrows():
            try:
                batdf = pd.read_csv(os.path.join(path,"bat","bat_%s.csv"%row.id))
                bowldf = pd.read_csv(os.path.join(path,"bowl","bow_%s.csv"%row.id))
                list_of_values.append(derive_batting_features(batdf,row.id)|derive_bowling_features(bowldf,row.id))
            except:
                pass
        tempdf = tempdf.merge(pd.DataFrame(list_of_values),left_on="id",right_on="id",how='left')
        finaldf = pd.concat([finaldf,tempdf])
finaldf.to_csv(r"C:\Users\Alok\Downloads\dataset\espn_data\complete_data\t20_2005_2021.csv")